## ANN and Financial model
### Terminalogies and formulas
* Re - cost of equity, percentage
* DPS - divident per share, value
* CMV - current market value of stock, value
* GRD - growth rate of divident, percentage (which is 0 for these banks)
* MMRP - mature market risk premium

#### Financial model:
```{math}
Re = (DPS / CMV) + GRD   
CMV = DPS / Re
Re = MMRP + CRP
CMV = DPS / (MMRP + CRP)
```

#### To get CMV from ANN model we will use:
```{math}
CMV = marketCap / shareOutstanding
```

In [28]:
import pandas as pd
import numpy as np
import yfinance as yf
from forex_python.converter import CurrencyRates

### Calling DF

In [8]:
turkey = pd.read_csv('turkey_predictions.csv')
developing = pd.read_csv('developing_predictions.csv')
tickers_turkey = turkey['symbol']
tickers_developing = developing['symbol']

In [9]:
developing

,symbol,shortName,marketCap,point_prediction_developing,lower_bound_developing,upper_bound_developing,country
0,HDFCBANK.NS,HDFC BANK,119.246483,82.549179,74.300049,90.798309,India
1,ICICIBANK.NS,ICICI BANK,67.968793,48.909954,40.255404,57.564504,India
2,SBIN.NS,STATE BK OF INDIA,55.169843,58.287987,45.688508,70.887466,India
3,KOTAKBANK.NS,KOTAK MAHINDRA BAN,54.205208,22.408054,14.680852,30.135257,India
4,AXISBANK.NS,AXIS BANK,33.720593,20.288666,15.627705,24.949627,India
...,...,...,...,...,...,...,...
123,VAKBN.IS,VAKIFLAR BANKASI,1.562017,5.386577,4.030570,6.742583,Turkey
124,HALKB.IS,T. HALK BANKASI,1.343121,4.381282,2.309190,6.453375,Turkey
125,TSKB.IS,T.S.K.B.,0.406916,1.100016,0.537522,1.662511,Turkey
126,ALBRK.IS,ALBARAKA TURK,0.238569,0.118892,-0.082575,0.320360,Turkey


### Constant variables

`mmrp` stands for Mature Market Risk Premium which is the risk premium associated with investing in a mature market like the USA. 

`crp` stands for Country Risk Premium associated with investing in an emerging market specific to each country, this value is added on top of `mmrp` to find cost of equity.

In [13]:
mmrp = 0.0472  # developed
india_crp = 0.0213
argentina_crp = 0.1162
brazil_crp = 0.0291
indonesia_crp = 0.0184
russia_crp = 0.0213
thailand_crp = 0.0155
turkey_crp = 0.0533   # turkey

developing_crp = [india_crp, argentina_crp, brazil_crp, indonesia_crp, russia_crp, thailand_crp, turkey_crp]

In [29]:
conditions = [(developing['country'] == 'India'),(developing['country'] == 'Argentina'),(developing['country'] == 'Brazil'),
             (developing['country'] == 'Indonesia'),(developing['country'] == 'Russia'),(developing['country'] == 'Thailand'),
             (developing['country'] == 'Turkey')]

choices = developing_crp
developing['cpr'] = np.select(conditions, choices)

#### Getting currencies

We write Argentina currency manually because its rate was not available at Forex Python

In [48]:
currencies = ['INR', 'BRL', 'IDR', 'RUB', 'THB', 'TRY']
c = CurrencyRates()
currency_rates = [c.get_rate(i, 'USD') for i in currencies]
currency_rates.insert(1, 0.010138552)
developing['currency'] = np.select(conditions, currency_rates)
currency_rates

[0.013552520498201708,
 0.010138552,
 0.18800936918436756,
 6.997625261910861e-05,
 0.013704765736097597,
 0.029895408163265302,
 0.11312868037455352]

In [51]:
developing

,symbol,shortName,marketCap,point_prediction_developing,lower_bound_developing,upper_bound_developing,country,cpr,currency
0,HDFCBANK.NS,HDFC BANK,119.246483,82.549179,74.300049,90.798309,India,0.0213,0.013553
1,ICICIBANK.NS,ICICI BANK,67.968793,48.909954,40.255404,57.564504,India,0.0213,0.013553
2,SBIN.NS,STATE BK OF INDIA,55.169843,58.287987,45.688508,70.887466,India,0.0213,0.013553
3,KOTAKBANK.NS,KOTAK MAHINDRA BAN,54.205208,22.408054,14.680852,30.135257,India,0.0213,0.013553
4,AXISBANK.NS,AXIS BANK,33.720593,20.288666,15.627705,24.949627,India,0.0213,0.013553
...,...,...,...,...,...,...,...,...,...
123,VAKBN.IS,VAKIFLAR BANKASI,1.562017,5.386577,4.030570,6.742583,Turkey,0.0533,0.113129
124,HALKB.IS,T. HALK BANKASI,1.343121,4.381282,2.309190,6.453375,Turkey,0.0533,0.113129
125,TSKB.IS,T.S.K.B.,0.406916,1.100016,0.537522,1.662511,Turkey,0.0533,0.113129
126,ALBRK.IS,ALBARAKA TURK,0.238569,0.118892,-0.082575,0.320360,Turkey,0.0533,0.113129


### Bank variables

In [53]:
def bank_variables(tickers):
    financial_model_developing = list()
    financial_model_developed = list()
    
    for i in range(0, len(tickers)):
        bank_info =  yf.Ticker(tickers[i]).info
        if(bank_info['trailingEps'] and bank_info['sharesOutstanding'] and bank_info['currentPrice']):
            print(i, bank_info['symbol'], end = ', ')
            dps = bank_info['trailingEps'] * developing.currency[i]
            market_cap = bank_info['marketCap'] / 10**9  * developing.currency[i]
            share_outstanding = bank_info['sharesOutstanding']  / 10**9
            real_cmv = bank_info['currentPrice'] * developing.currency[i]
        
            # Calculating CMV based on financial models
            re1 = mmrp + developing.cpr[i]
            cmv1 = dps / re1           

            # Converting CMV1 to MarketCap
            marketCap_developing = cmv1 * share_outstanding

            # adding to list
            financial_model_developing.append(marketCap_developing)

            # Estimating what would be in developed markets CMV based on financial models
            re2 = mmrp
            cmv2 = dps / re2

            # Converting CMV2 to MarketCap
            marketCap_developed = cmv2 * share_outstanding

            financial_model_developed.append(marketCap_developed)
    
        
    return financial_model_developing, financial_model_developed


marketcaps_developing = bank_variables(tickers_developing)


marketcaps_developing


0 HDFCBANK.NS, 1 ICICIBANK.NS, 2 SBIN.NS, 3 KOTAKBANK.NS, 4 AXISBANK.NS, 5 INDUSINDBK.NS, 6 BANDHANBNK.NS, 7 PNB.NS, 8 IDBI.NS, 9 IOB.NS, 10 AUBANK.NS, 11 YESBANK.NS, 12 IDFCFIRSTB.NS, 13 CANBK.NS, 14 UNIONBANK.NS, 15 BANKINDIA.NS, 16 BANKBARODA.NS, 17 CENTRALBK.NS, 18 FEDERALBNK.NS, 19 INDIANB.NS, 20 UCOBANK.NS, 21 MAHABANK.NS, 22 CUB.NS, 23 RBLBANK.NS, 24 PSB.NS, 25 EQUITASBNK.NS, 26 CSBBANK.NS, 27 KARURVYSYA.NS, 28 UJJIVANSFB.NS, 29 DCBBANK.NS, 31 KTKBANK.NS, 32 SOUTHBANK.NS, 33 SURYODAY.NS, 35 DHANBANK.NS, 36 GGAL.BA, 37 BMA.BA, 38 SUPV.BA, 39 BHIP.BA, 40 ITUB4.SA, 41 ITUB3.SA, 42 BBDC4.SA, 43 BBDC3.SA, 45 ITSA4.SA, 46 ITSA3.SA, 47 BBAS3.SA, 48 BIDI3.SA, 49 BIDI4.SA, 50 BPAN4.SA, 51 BRSR6.SA, 52 ABCB4.SA, 53 BMGB4.SA, 54 PINE4.SA, 57 BBCA.JK, 58 BBRI.JK, 59 BMRI.JK, 60 ARTO.JK, 61 BBNI.JK, 62 BRIS.JK, 63 BNLI.JK, 64 AGRO.JK, 65 BANK.JK, 66 BBHI.JK, 67 BDMN.JK, 68 BNII.JK, 69 BTPS.JK, 70 BINA.JK, 71 BNGA.JK, 72 BSIM.JK, 73 BBSI.JK, 74 PNBN.JK, 75 NISP.JK, 76 BBTN.JK, 77 BBKP.JK, 78 

([64.96498816139595,
  39.3315217877238,
  49.485714272963094,
  19.606136624396576,
  16.848718648233675,
  6.981983035641513,
  4.015588300246575,
  6.936323433800551,
  3.946197735178893,
  1.9072896415317424,
  2.343196978697144,
  -8.179111396046155,
  1.0683641115688713,
  6.22608742697665,
  6.04448988388025,
  4.6460159997584025,
  3.3957953095240074,
  -2.9712714841462375,
  3.2821014030835016,
  6.2295763335328305,
  0.4021280054481641,
  1.1810962986256097,
  0.759811049402239,
  1.1453033843306575,
  -28.632586928996083,
  0.7888458562187362,
  0.4321352214714067,
  0.7116456069737906,
  0.01709704884604914,
  0.6574737180017873,
  0.9460653659363039,
  0.14077440068126768,
  0.027506988969678618,
  0.07358477321091657,
  0.8804753961867027,
  1.8451244839151664,
  0.1629765160591628,
  8.730087379979909e-05,
  25.719925856118504,
  26.316769122399048,
  19.461820319223303,
  25.267974601407605,
  13.345662356526981,
  5.97435097467904,
  33.347488047296885,
  0.02260251959

#### Some tickers are skipped because they don't have information for either trailingEps or sharesOutstanding, so we need to drop rows with these tickers

In [64]:
skipped = [30, 34, 44, 55, 56]
for i in skipped:
    developing = developing.drop(index = i)

In [68]:
developing

,symbol,shortName,marketCap,point_prediction_developing,lower_bound_developing,upper_bound_developing,country,cpr,currency
0,HDFCBANK.NS,HDFC BANK,119.246483,82.549179,74.300049,90.798309,India,0.0213,0.013553
1,ICICIBANK.NS,ICICI BANK,67.968793,48.909954,40.255404,57.564504,India,0.0213,0.013553
2,SBIN.NS,STATE BK OF INDIA,55.169843,58.287987,45.688508,70.887466,India,0.0213,0.013553
3,KOTAKBANK.NS,KOTAK MAHINDRA BAN,54.205208,22.408054,14.680852,30.135257,India,0.0213,0.013553
4,AXISBANK.NS,AXIS BANK,33.720593,20.288666,15.627705,24.949627,India,0.0213,0.013553
...,...,...,...,...,...,...,...,...,...
123,VAKBN.IS,VAKIFLAR BANKASI,1.562017,5.386577,4.030570,6.742583,Turkey,0.0533,0.113129
124,HALKB.IS,T. HALK BANKASI,1.343121,4.381282,2.309190,6.453375,Turkey,0.0533,0.113129
125,TSKB.IS,T.S.K.B.,0.406916,1.100016,0.537522,1.662511,Turkey,0.0533,0.113129
126,ALBRK.IS,ALBARAKA TURK,0.238569,0.118892,-0.082575,0.320360,Turkey,0.0533,0.113129


In [69]:
# What the financial model implies through ratios

new_fin_dev = (developing['marketCap'] * marketcaps_developing[1]) / marketcaps_developing[0]

In [70]:
developing['financial_model_developed'] = new_fin_dev

In [72]:
# What the financial model implies through ratios

new_fin_developing = (developing['marketCap'] * developing['point_prediction_developing']) / developing['financial_model_developed']

In [73]:
developing['financial_model_developing'] = new_fin_developing

In [74]:
developing['undervalued'] = developing['marketCap'] < developing['financial_model_developing']

In [75]:
developing

,symbol,shortName,marketCap,point_prediction_developing,lower_bound_developing,upper_bound_developing,country,cpr,currency,financial_model_developed,financial_model_developing,undervalued
0,HDFCBANK.NS,HDFC BANK,119.246483,82.549179,74.300049,90.798309,India,0.0213,0.013553,173.058985,56.880602,False
1,ICICIBANK.NS,ICICI BANK,67.968793,48.909954,40.255404,57.564504,India,0.0213,0.013553,98.641151,33.701457,False
2,SBIN.NS,STATE BK OF INDIA,55.169843,58.287987,45.688508,70.887466,India,0.0213,0.013553,80.066404,40.163401,False
3,KOTAKBANK.NS,KOTAK MAHINDRA BAN,54.205208,22.408054,14.680852,30.135257,India,0.0213,0.013553,78.666457,15.440294,False
4,AXISBANK.NS,AXIS BANK,33.720593,20.288666,15.627705,24.949627,India,0.0213,0.013553,48.937725,13.979927,False
...,...,...,...,...,...,...,...,...,...,...,...,...
123,VAKBN.IS,VAKIFLAR BANKASI,1.562017,5.386577,4.030570,6.742583,Turkey,0.0533,0.113129,3.325904,2.529815,True
124,HALKB.IS,T. HALK BANKASI,1.343121,4.381282,2.309190,6.453375,Turkey,0.0533,0.113129,2.859824,2.057677,True
125,TSKB.IS,T.S.K.B.,0.406916,1.100016,0.537522,1.662511,Turkey,0.0533,0.113129,0.866422,0.516625,True
126,ALBRK.IS,ALBARAKA TURK,0.238569,0.118892,-0.082575,0.320360,Turkey,0.0533,0.113129,0.507971,0.055838,False


In [85]:
undervalued_developing_df = developing.drop(developing[developing['undervalued'] == False].index)
undervalued_developing_df.shape

(21, 12)

In [87]:
undervalued_developing_df.to_csv('undervalued_developing.csv', index = False)

In [86]:
undervalued_developing_df

,symbol,shortName,marketCap,point_prediction_developing,lower_bound_developing,upper_bound_developing,country,cpr,currency,financial_model_developed,financial_model_developing,undervalued
13,CANBK.NS,CANARA BANK,3.650832,5.744370,3.184956,8.303785,India,0.0213,0.013553,5.298347,3.958165,True
14,UNIONBANK.NS,UNION BK OF INDIA,3.340243,5.025617,3.560555,6.490679,India,0.0213,0.013553,4.847599,3.462907,True
19,INDIANB.NS,INDIAN BANK,2.107057,4.918976,3.979053,5.858898,India,0.0213,0.013553,3.057911,3.389426,True
47,BBAS3.SA,BRASIL ON NM,15.679843,37.345146,26.887217,47.803075,Brazil,0.0291,0.188009,25.346865,23.102109,True
74,PNBN.JK,Bank Pan Indonesia Tbk,1.294616,2.012228,1.522097,2.502359,Indonesia,0.0184,0.000070,1.799297,1.447823,True
85,SDRA.JK,Bank Woori Saudara Indonesia 19,0.342228,0.562714,0.177144,0.948283,Indonesia,0.0184,0.000070,0.475638,0.404879,True
100,VTBR.ME,VTB BANK PJSC,9.148014,22.631565,19.522033,25.741097,Russia,0.0213,0.013705,13.276249,15.594305,True
102,BSPB.ME,BANK ST PETERSBURG,0.506606,1.398919,0.904654,1.893184,Russia,0.0213,0.013705,0.735222,0.963927,True
103,USBN.ME,BANK URALSIB PJSC,0.437479,0.751779,0.331349,1.172209,Russia,0.0213,0.013705,0.634900,0.518014,True
104,KUZB.ME,KUZNETSKY BANK PUB,0.007255,0.028478,-0.143692,0.200648,Russia,0.0213,0.013705,0.010528,0.019623,True


### Extract Turkey

In [90]:
undervalued_turkey_df = undervalued_developing_df.drop(undervalued_developing_df[undervalued_developing_df['country'] != 'Turkey'].index)
undervalued_turkey_df

,symbol,shortName,marketCap,point_prediction_developing,lower_bound_developing,upper_bound_developing,country,cpr,currency,financial_model_developed,financial_model_developing,undervalued
118,GARAN.IS,GARANTI BANKASI,4.585135,9.839768,7.636162,12.043374,Turkey,0.0533,0.113129,9.762841,4.621264,True
119,AKBNK.IS,AKBANK,3.355317,8.856296,7.538009,10.174582,Turkey,0.0533,0.113129,7.144266,4.159375,True
120,ISCTR.IS,IS BANKASI (C),2.761096,8.548029,5.645167,11.450891,Turkey,0.0533,0.113129,5.879028,4.014597,True
121,YKBNK.IS,YAPI VE KREDI BANK.,2.406071,7.097673,5.773831,8.421515,Turkey,0.0533,0.113129,5.123096,3.333434,True
122,SAHOL.IS,SABANCI HOLDING,2.291788,7.013840,4.849965,9.177715,Turkey,0.0533,0.113129,4.879760,3.294062,True
123,VAKBN.IS,VAKIFLAR BANKASI,1.562017,5.386577,4.030570,6.742583,Turkey,0.0533,0.113129,3.325904,2.529815,True
124,HALKB.IS,T. HALK BANKASI,1.343121,4.381282,2.309190,6.453375,Turkey,0.0533,0.113129,2.859824,2.057677,True
125,TSKB.IS,T.S.K.B.,0.406916,1.100016,0.537522,1.662511,Turkey,0.0533,0.113129,0.866422,0.516625,True


In [91]:
undervalued_turkey_df.to_csv('undervalued_turkey.csv', index = False)